In [30]:
### preparing model for extracted features from the payloads

import pandas as pd


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier , VotingClassifier

# appending path for importing common process ua

import sys 
sys.path.append('/home/th3h04x/Documents/github/ProjectHeatBlast')


from ReverseProxy.utils.util import process_ua


payloads_with_feature_df = pd.read_csv("build/combined_payloads_with_features.csv" , index_col=0)
payloads_df = pd.read_csv("build/combined_payloads.csv" , index_col=0)


In [31]:
#dropping payload column

payloads_with_feature_df.drop(["payloads"],axis=1,inplace=True)

payloads_df

,payloads,is_malicious
0,Nokia,0
1,Sony,0
2,BMW,0
3,Palm,0
4,Adobe,0
...,...,...
41575,<form><button formaction=javascript&colon;aler...,1
41576,"<math><a xlink:href=""//jsfiddle.net/t846h/"">click",1
41577,"<object data=data:text/html;base64,PHN2Zy9vbmx...",1
41578,"<iframe src=""data:text/html,%3C%73%63%72%69%70...",1


In [32]:
import ipyparallel as ipp

cluster = ipp.Cluster(n=5)

await cluster.start_cluster()

Starting 5 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


<Cluster(cluster_id='1640844115-ndtu', profile='default', controller=<running>, engine_sets=['1640844116'])>

In [33]:
# getting x and y values

x = payloads_with_feature_df.drop(["is_malicious"],axis=1).values 
y = payloads_with_feature_df["is_malicious"].values

# Hyper Paramater Tuning for each custom feature models

In [34]:
# k neigherest neighbours

from sklearn.model_selection import GridSearchCV

knn_clf = GridSearchCV(KNeighborsClassifier() , {
        'n_neighbors':[10,50,100,150,200,250,500]
        } ,
        cv=5)

knn_clf.fit(x,y)
pd.DataFrame(knn_clf.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.050770,0.028584,0.219265,0.024089,10,{'n_neighbors': 10},0.997715,0.998918,0.996994,0.999519,0.996873,0.998004,0.001050,1
1,0.032193,0.000457,0.265185,0.014796,50,{'n_neighbors': 50},0.996633,0.998557,0.996272,0.999639,0.997475,0.997715,0.001243,2
2,0.033726,0.000493,0.346127,0.011961,100,{'n_neighbors': 100},0.996633,0.997956,0.994829,0.999399,0.995671,0.996898,0.001627,3
3,0.035850,0.003823,0.487658,0.041852,150,{'n_neighbors': 150},0.996513,0.997835,0.994228,0.999158,0.995430,0.996633,0.001736,4
4,0.034459,0.002850,0.544248,0.034341,200,{'n_neighbors': 200},0.995551,0.997354,0.993146,0.999158,0.995190,0.996080,0.002039,5
5,0.035396,0.001770,0.620059,0.022432,250,{'n_neighbors': 250},0.994589,0.997114,0.992304,0.998918,0.994949,0.995575,0.002263,6
6,0.034745,0.001867,1.158534,0.044737,500,{'n_neighbors': 500},0.991582,0.996753,0.990741,0.997715,0.993506,0.994060,0.002760,7


In [35]:
#Multinomial naive bayes

mnb_clf = GridSearchCV( MultinomialNB() , {
    "alpha":[0.1,0.3,0.5,0.7,1,2]
},cv=5)

mnb_clf.fit(x,y)
pd.DataFrame(mnb_clf.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.007337,0.001768,0.000980,0.000146,0.1,{'alpha': 0.1},0.990981,0.990741,0.984367,0.992424,0.990139,0.989731,0.002785,1
1,0.006927,0.000734,0.001040,0.000445,0.3,{'alpha': 0.3},0.990981,0.990741,0.984367,0.992424,0.990139,0.989731,0.002785,1
2,0.007764,0.002791,0.001890,0.000982,0.5,{'alpha': 0.5},0.990981,0.990741,0.984367,0.992424,0.990139,0.989731,0.002785,1
3,0.008651,0.002145,0.001306,0.000574,0.7,{'alpha': 0.7},0.990981,0.990741,0.984367,0.992424,0.990139,0.989731,0.002785,1
4,0.008467,0.002017,0.000886,0.000207,1,{'alpha': 1},0.990981,0.990741,0.984367,0.992424,0.990139,0.989731,0.002785,1
5,0.006779,0.001401,0.000901,0.000140,2,{'alpha': 2},0.990981,0.990741,0.984367,0.992424,0.990019,0.989707,0.002781,6


In [36]:
#support vector machine

svc_clf = GridSearchCV(SVC() , {
    "kernel":["rbf","linear","sigmoid"] ,
    "gamma":[0.1,0.5,1,2],
    "C":[1,10,20]
},cv=5)

svc_clf.fit(x,y)
pd.DataFrame(svc_clf.cv_results_)

In [ ]:
#Decision Tree Classifier

dt_clf = GridSearchCV(DecisionTreeClassifier(),{
    "criterion":["gini","entropy"],
    "max_depth":[10,20,30]
},cv=5)

dt_clf.fit(x,y)

pd.DataFrame(dt_clf.cv_results_)



In [ ]:
#random forest classifier

rf_clf = GridSearchCV(RandomForestClassifier(),{
    "criterion":["gini","entropy"],
    "max_depth":[10,20,30] ,
    "n_estimators":[20,30,50]
},cv=5)

rf_clf.fit(x,y)
pd.DataFrame(rf_clf.cv_results_)

In [ ]:
#logistic regresion

lg_clf = GridSearchCV(LogisticRegression() , {
    "C":[1,5,10]
} ,cv=5)

lg_clf.fit(x,y)
pd.DataFrame(lg_clf.cv_results_)
